In [1]:
import faiss
import torch
import json
from transformers import AutoTokenizer, AutoModel
from torchvision import models, transforms
from PIL import Image
import numpy as np
import os

In [ ]:
# Initialize models
text_model = AutoModel.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")
tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")


image_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])


image_dir = r"D:\Sem7-project-folders\augmented_dataset\augmented_dataset"
json_path = r"D:\phase2\aj-sanskrit\aj-sanskrit\description_sanskrit.json"

text_dim = 384  
index = faiss.IndexFlatL2(text_dim)  # Using L2 distance metric for text embeddings
metadata = {}  

with open(json_path, "r", encoding="utf-8") as f:
    data = json.load(f)

for entry in data:
    filename = entry["filename"]
    description = entry["description"]
    image_path = os.path.join(image_dir, filename)
    
    
    if not os.path.exists(image_path):
        print(f"Image file {filename} not found. Skipping.")
        continue
    
    inputs = tokenizer(description, return_tensors="pt", max_length=512, truncation=True)
    with torch.no_grad():
        text_embedding = text_model(**inputs).last_hidden_state.mean(dim=1).squeeze().numpy()  # Shape: (384,)

    # Add text embedding to Faiss index and map to image path in metadata
    index.add(text_embedding.reshape(1, -1))
    metadata[len(metadata)] = image_path

# Save the Faiss index and metadata
faiss.write_index(index, "Karana.index")
torch.save(metadata, "metadata.pth")
print("Indexing completed and saved.")


Indexing completed and saved.


In [5]:
import torch

# Path to your metadata.pth file
metadata_path = "metadata.pth"

# Load the .pth file
metadata = torch.load(metadata_path)

# Inspect the keys in the file
print("Keys in the metadata file:", metadata.keys())

# Example: Print specific content
for key, value in metadata.items():
    print(f"\nKey: {key}")
    print(f"Value: {value}")


Keys in the metadata file: dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45])

Key: 0
Value: D:\Sem7-project-folders\augmented_dataset\augmented_dataset\Talapushpaputam\img1.jpg

Key: 1
Value: D:\Sem7-project-folders\augmented_dataset\augmented_dataset\Leenam\img1.jpg

Key: 2
Value: D:\Sem7-project-folders\augmented_dataset\augmented_dataset\Vartitam\img1.jpg

Key: 3
Value: D:\Sem7-project-folders\augmented_dataset\augmented_dataset\Valitorukam\img1.jpg

Key: 4
Value: D:\Sem7-project-folders\augmented_dataset\augmented_dataset\Mandalaswastikam\img1.jpg

Key: 5
Value: D:\Sem7-project-folders\augmented_dataset\augmented_dataset\ArdhaSwastikam\img1.jpg

Key: 6
Value: D:\Sem7-project-folders\augmented_dataset\augmented_dataset\Dikswastikam\img1.jpg

Key: 7
Value: D:\Sem7-project-folders\augmented_dataset\augmented_dataset\Prasthaswastikam\img1.jpg

Key: 8
V